d:\Users\llizh\miniconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...


In [1]:

from transformers import AutoProcessor, AutoModelForVision2Seq

from utils.ours import evolve_ours_greedy
print("Loading model...")# Load model directly

min_pixels = 256 * 28 * 28
max_pixels = 512 * 28 * 28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct",cache_dir='D:/cache/huggingface/transformers',trust_remote_code=True,
                                                min_pixels=min_pixels, max_pixels=max_pixels)
model = AutoModelForVision2Seq.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct",cache_dir='D:/cache/huggingface/transformers',
        trust_remote_code=True,
        device_map="cuda",
    )

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)



d:\Users\llizh\miniconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]


In [2]:

evolve_ours_greedy()

generated_ids = model.generate(**inputs, max_new_tokens=512, top_p=None, top_k=None, do_sample=False, alpha=0.6, threshold_top_p=0.9, threshold_top_k=20,
                            early_exit_layers=[i for i in range(25,35)], return_dict_in_generate=True,
        output_hidden_states=True)

generated_ids = generated_ids.sequences

D:\Code\Efficient-HA\transformers-4.49.0\src\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-06` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [4]:
outputs = model.generate(**inputs, max_new_tokens=512,return_dict_in_generate=True,output_hidden_states=True)


In [3]:
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

In [4]:
print(output_text
      )

The candy in your hand has an animal design on it. The animal appears to be a fish, as indicated by the shape and markings on the candy.
